In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

In [8]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,5 mins 59 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 18 days
H2O cluster name:,H2O_from_python_marta_3svtha
H2O cluster total nodes:,1
H2O cluster free memory:,1.700 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [9]:
train = h2o.import_file('dataset/diamonds_train.csv')
test = h2o.import_file('dataset/diamonds_test.csv')
sub_data = h2o.import_file('dataset/sample_submission.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
y = "price"
x = train.columns
x.remove(y)

In [7]:
run_automl_for_seconds = 18000
#weight_col = "carat"
balance_classes= True #busca las variables que tienen más peso. Lo tiene en cuenta.
aml = H2OAutoML(max_runtime_secs = run_automl_for_seconds, #corremos el modelo
                #weights_column = weight_col,
                balance_classes = True,
                sort_metric = "RMSE", #lo que nos piden para la competición
                include_algos = ["GBM", "XGBoost"] #GradientBoost #XGBoost
               )

In [10]:
train_final, valid = train.split_frame(ratios=[0.8]) #split
#train_final es el train y valid es el test. Entrena y valida el df con la misma función, de una.

In [11]:
aml.train(x=x, y =y, training_frame=train_final, validation_frame=valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
leader_model = aml.leader
pred = leader_model.predict(test_data=test)
pred

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
2960.22
5524.38
9684.67
3974.21
1633.49
6240.72
4820.01
4227.7
5204.59
14981.1


In [13]:
pred_pd = pred.as_data_frame() #predicción que acabamos de sacar
sub = sub_data.as_data_frame() #sample_submission para mantener el modelo

In [14]:
perf = aml.leader.model_performance(valid)
perf


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 289684.7175405213
RMSE: 538.2236686922281
MAE: 274.2278305472553
RMSLE: 0.09236932222065948
Mean Residual Deviance: 289684.7175405213


In [15]:
sub['price'] = pred_pd
h2o_test = pd.DataFrame(sub.price)
h2o_test.index.name = "id"

In [16]:
display(h2o_test)

,price
id,
0,2960.218395
1,5524.383437
2,9684.666094
3,3974.210518
4,1633.488771
5,6240.722179
6,4820.011542
7,4227.697188
8,5204.592410


AttributeError: 'NoneType' object has no attribute 'head'

In [17]:
h2o_test.to_csv('output/result.csv')

<a href="resultado_h2o_balanceclassestrue_18000.csv"> Download File </a>